In [2]:
from Qommunity.samplers.hierarchical.advantage_sampler import AdvantageSampler
from Qommunity.searchers.hierarchical_community_searcher import HierarchicalCommunitySearcher

In [3]:
import numpy as np
import networkx as nx
import os


def make_dir(dir):
    if not os.path.exists(dir):
        os.mkdir(dir)



MIN_NODES = 10
MAX_NODES = 100

num_nodes = np.linspace(MIN_NODES, MAX_NODES, MAX_NODES//MIN_NODES)


dir = "networks/powerlaw_m=1_p=0.2"
os.makedirs(os.path.dirname(dir+"/"), exist_ok=True)

solver = "advantage"


try:
    Graphs = np.load(f"{dir}/graphs.npy", allow_pickle=True)
except:
    Graphs = np.empty(shape=(len(num_nodes),), dtype=object)
    for i, n in enumerate(num_nodes):
        Graphs[i] = nx.powerlaw_cluster_graph(n=n, m=1, p=0.2)
    np.save(f"{dir}/graphs.npy", Graphs)




dir = "profiling"
os.makedirs(os.path.dirname(dir+"/"), exist_ok=True)
make_dir(f"{dir}/{solver}")

In [12]:
Graphs = np.load("networks/powerlaw_m=1_p=0.2/graphs.npy", allow_pickle=True)

In [9]:
N_RUNS = 1

In [18]:
G_idx = MAX_NODES//MIN_NODES - 1
print(G_idx)
Graphs[G_idx:]

9


array([<networkx.classes.graph.Graph object at 0x00000200F738E290>],
      dtype=object)

In [4]:
from tqdm import tqdm


N_RUNS = 1

G_idx = 1

def run_adv():
    mods = np.zeros((num_nodes[:G_idx].shape[0], N_RUNS))
    comms = np.empty((num_nodes[:G_idx].shape[0], N_RUNS), dtype=object)

    mods_graph_N = np.zeros((N_RUNS))
    comms_graph_N = np.empty((N_RUNS), dtype=object)
    for i, G in tqdm(enumerate(Graphs[:G_idx])):
        current_net_size = int(G.number_of_nodes())    
        for r in range(N_RUNS):
            # new instance of the sampler each time
            advantage = AdvantageSampler(G, num_reads=100)
            hierch_searcher = HierarchicalCommunitySearcher(advantage)
            comms_res = hierch_searcher.hierarchical_community_search()
            
            mod_score = nx.community.modularity(G, comms_res)
            mods_graph_N[r] = mod_score
            comms_graph_N[r] = comms_res

            try:
                np.save(f"{dir}/{solver}/{solver}-network_size_{current_net_size}", mods_graph_N)
                np.save(f"{dir}/{solver}/{solver}-network_size_{current_net_size}_comms", comms_graph_N)
            except Exception as e:
                print(f"iter: {i} run: {r} saving failed")
                print(e)

        # current_net_size = int(num_nodes[i])
        current_net_size = int(G.number_of_nodes())
        mods[i] = mods_graph_N
        comms[i] = comms_graph_N
        np.save(f"{dir}/{solver}/{solver}-network_size_{current_net_size}", mods_graph_N)
        np.save(f"{dir}/{solver}/{solver}-network_size_{current_net_size}_comms", comms_graph_N)

    np.save(f"{dir}/{solver}/{solver}-mods_final", mods)
    np.save(f"{dir}/{solver}/{solver}-comms_final", comms)

In [4]:
%load_ext line_profiler

In [10]:
from QHyper.solvers.quantum_annealing.advantage import Advantage, OptimizerFunction

In [6]:
np.arange(10)[9]

9

In [12]:
%lprun -f OptimizerFunction.run_advantage run_adv()

1it [00:08,  8.12s/it]


Timer unit: 1e-07 s

Total time: 0.565313 s
File: C:\Users\basia\Desktop\Praca_Inzynierska_2024\QHyper\QHyper\solvers\quantum_annealing\advantage.py
Function: run_advantage at line 51

Line #      Hits         Time  Per Hit   % Time  Line Contents
    51                                               def run_advantage(self, args: npt.NDArray) -> SampleSet:
    52         5       5548.0   1109.6      0.1          qubo = Converter.create_qubo(self.problem, args)
    53         5       2743.0    548.6      0.0          qubo_terms, offset = convert_qubo_keys(qubo)
    54         5      10190.0   2038.0      0.2          bqm = BinaryQuadraticModel.from_qubo(qubo_terms, offset=offset)
    55        10    5634609.0 563460.9     99.7          sampleset = self.embedding_compose.sample(
    56         5         22.0      4.4      0.0              bqm, num_reads=self.num_reads, chain_strength=self.chain_strength
    57                                                   )
    58                     